# 네이버 기사 데이터 수집
- scrapy에서 selenium 사용해보기

## 1. 프로젝트 생성

In [1]:
!scrapy startproject naver_article

New Scrapy project 'naver_article', using template directory '/opt/anaconda3/lib/python3.8/site-packages/scrapy/templates/project', created in:
    /Users/dokyum/Workspace/DSS - /TIL-data-science/scrapy/naver_article

You can start your first spider with:
    cd naver_article
    scrapy genspider example example.com


In [2]:
!tree naver_article/

naver_article/
├── naver_article
│   ├── __init__.py
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       └── __init__.py
└── scrapy.cfg

2 directories, 7 files


## 2. xpath 찾기
- article links : [not()] 사용
- article content

In [3]:
import scrapy, requests
from scrapy.http import TextResponse
from selenium import webdriver

In [53]:
url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101"
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
req = requests.get(url, headers=headers)
req

<Response [200]>

In [54]:
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [55]:
links = response.xpath('//*[@id="main_content"]/div[2]/ul/li/dl/dt[not(@class="photo")]/a/@href').extract()

len(links), links[0]

(20,
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=009&aid=0004728974')

In [14]:
# selenium 사용 : xpath 가능

In [56]:
options = webdriver.ChromeOptions()
options.add_argument("headless")
driver = webdriver.Chrome(options=options)

In [57]:
driver.get(url)

In [58]:
elements = driver.find_elements_by_xpath('//*[@id="main_content"]/div[2]/ul/li/dl/dt[not(@class="photo")]/a')
len(elements)

20

In [59]:
links = [element.get_attribute("href") for element in elements]
len(links), links[0]

(20,
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=009&aid=0004728974')

In [60]:
driver.quit()

In [20]:
## content : title, category, content

In [61]:
link = links[0]
link

'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=009&aid=0004728974'

In [62]:
req = requests.get(link, headers=headers)
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [63]:
title = response.xpath('//*[@id="articleTitle"]/text()').extract()[0]
category = response.url.split('sid1=')[1].split('&')[0]
content = response.xpath('//*[@id="articleBodyContents"]/text()').extract()
content = " ".join(content).strip()
title, category, content

('"노동·환경·디지털…바이든시대, 무역의 모든 룰이 바뀐다"',
 '101',
 '◆ 매경·한미경제학회 포럼 / 한국경제 정책포럼 ◆    "코로나19 이후 경기가 자연적으로 회복되기만을 기대해선 안 된다. 뼈를 깎는 각오로 근본 체질을 개혁하지 않으면 한국 경제 침체는 계속될 것이다." 길고 길었던 코로나19 터널도 올 들어 끝이 보일 것이란 전망이 나온다. 그렇다면 한국 경제도 자연스러운 경기회복을 기대할 수 있을 것인가. 매일경제와 한미경제학회(KAEA)가 5일(현지시간) 공동 주최한 \'한국 경제 정책포럼\'에 참가한 석학들은 "그렇지 않다"고 잘라 말했다. 정부가 \'자연스러운 경기회복의 환상\'에 빠져 있으면 안 된다는 것이다. 기조발표에 나선 김흥종 대외경제정책연구원(KIEP) 원장·장지상 산업연구원(KIET) 원장·최정표 한국개발연구원(KDI) 원장은 △조 바이든 차기 미국 행정부 하 통상질서 변화 △국내 제조업 구조적 위기 △불평등이 심화된 K자 회복 양상 등을 직시하고 정부가 올해부터 구조조정과 체질 대전환을 냉철히 추진해 나가야 한다고 지적했다. 바이든 행정부 시대 무역환경에 대해 다룬 김 원장은 노동·환경·디지털 세 분야의 \'삼각 파도\'가 곧 한국 경제를 덮칠 것으로 내다봤다. 김 원장은 "현재 한미 자유무역협정(FTA)은 신북미무역협정(USMCA), 미·일 FTA 등과 비교했을 때 상당히 낮은 수준의 무역협정이 됐다"며 "노동·환경 등에서 USMCA 수준에 준하는 새로운 FTA를 바이든 행정부가 내놓을 것"이라고 예상했다. 그러면서 "정부는 그전에 각국에 국제협력을 제안해 그린산업 분야 관련 기술규제 등에 미리 대비해야 한다"고 지적했다. 또 그는 "한국의 디지털 통상환경이 낙후돼 유럽연합(EU) 개인정보보호 규정(GDPR) 등에 제대로 대처하지 못하고 있어 업그레이드가 시급하다"며 "얼마 전 통과된 \'데이터3법\'에 부족한 개인정보 국외이전 관련 규정을 보완하거나 디지털 통상 관련 통계를 구축하는 작업이 필요하다"고 강조했다. 

## 3. items.py

In [ ]:
# %load naver_article/naver_article/items.py

In [33]:
%%writefile naver_article/naver_article/items.py
import scrapy


class NaverArticleItem(scrapy.Item):
    title = scrapy.Field()
    content = scrapy.Field()
    category = scrapy.Field()
    link = scrapy.Field()

Overwriting naver_article/naver_article/items.py


## 4. spider.py

In [83]:
%%writefile naver_article/naver_article/spiders/spiders.py
import scrapy
from selenium import webdriver
from naver_article.items import NaverArticleItem


class ArticleSpider(scrapy.Spider):
    name = "NaverArticle"
    allow_domain =["https://news.naver.com"]
    
    def start_requests(self):
        url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101"
        
        #selenium
        options = webdriver.ChromeOptions()
        options.add_argument("headless")
        driver = webdriver.Chrome(options=options)

        driver.get(url)
        print("this is urls : {}".format(url))
        ## 오류찾기
        
        headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
        
        elements = driver.find_elements_by_xpath('//*[@id="main_content"]/div[2]/ul/li/dl/dt[not(@class="photo")]/a')
        links = [element.get_attribute("href") for element in elements]
#         print("this is links : {}".format(links))
        
        for link in links:
            yield scrapy.Request(link, headers=headers, callback=self.parse_content)
            
    def parse_content(self, response):
        item = NaverArticleItem()
        item['title'] = response.xpath('//*[@id="articleTitle"]/text()').extract()[0]
        item['category'] = response.url.split('sid1=')[1].split('&')[0]
        content = response.xpath('//*[@id="articleBodyContents"]/text()').extract() 
        item['content'] =" ".join(content).strip()
        item['link'] = response.url
        yield item

Overwriting naver_article/naver_article/spiders/spiders.py


## 5. scrapy crawl

In [36]:
!pwd

/Users/dokyum/Workspace/DSS - /TIL-data-science/scrapy


In [ ]:
!cd naver_article && scrapy crawl NaverArticle -o article.csv

In [82]:
!cd naver_article && scrapy crawl NaverArticle --logfile NaverArticle.log

In [86]:
df = pd.read_csv('./naver_article/article.csv')
df.tail(2)

,category,content,link,title
18,101,지난해 서울 강남3구 아파트 매매 4건 중 1건은 외지인이 사들인 '상경투자'인 것...,https://news.naver.com/main/read.nhn?mode=LSD&...,작년 강남 아파트 4곳 중 1곳은 외지인이 샀다
19,101,경기도 의정부시 장암5구역 재개발 사업 수주전에서 SK건설과 현대엔지니어링 컨소시엄...,https://news.naver.com/main/read.nhn?mode=LSD&...,"의정부 장암5구역 수주전 치열.. SK컨소, 이주비 대출 80% 지원"


## 6. pipelines
- 크롤링한 데이터를 mongodb에 저장하기 : pymongo 사용
- 크롤링한 데이터에서 특정한 키워드가 있는 기사가 수집되면 slack 메신저로 기사 내용과 링크 전송하기
    - slack 메시지의 incoming webhook은 1초에 1번 사용가능 > time.sleep(1) 사용

In [ ]:
#pymongo 사용

In [48]:
!pip install pymongo

     |████████████████████████████████| 380 kB 2.0 MB/s eta 0:00:01


In [49]:
import pymongo

In [ ]:
client = pymongo.MongoClient("mongodb://")
client

In [67]:
db = client.naver_article
collection = db.article
collection.insert({"title": "scrapy"})

<ipython-input-67-1a86da07e8a4>:3: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  collection.insert({"title": "scrapy"})


ObjectId('5ff57d03f62040bb9a17a721')

In [68]:
%%writefile naver_article/naver_article/mongodb.py
import pymongo

client = pymongo.MongoClient("mongodb://")
db = client.naver_article
collection = db.article

Writing naver_article/naver_article/mongodb.py


In [69]:
# pipeline 추가

In [ ]:
# %load naver_article/naver_article/pipelines.py

In [71]:
%%writefile naver_article/naver_article/pipelines.py
from itemadapter import ItemAdapter
from .mongodb import collection

class NaverArticlePipeline:
    def process_item(self, item, spider):
        
        data = { "title" : item["title"], "category" : item["category"],"content" : item["content"],"link" : item["link"] }
        collection.insert(data)
        
        return item


Overwriting naver_article/naver_article/pipelines.py


## pipeline 사용 설정 : settings.py
```
ITEM_PIPELINES = {
   'naver_article.pipelines.NaverArticlePipeline': 300,
}
```

In [ ]:
!cd naver_article && scrapy crawl NaverArticle